<a href="https://colab.research.google.com/github/gaussssss/machinelearning-tpe/blob/main/Random_Forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
importations des packages python nécessaires
"""
import pandas as pd
import numpy as np
from pprint import pprint
import scipy.stats as sps
from google.colab import drive
drive.mount('/content/drive/')
data_path = "/content/drive/MyDrive/Colab Notebooks/random_forest/"
dataset = pd.read_csv(data_path+'agaricus-lepiota.data',header=None)
dataset = dataset.sample(frac=1)
dataset.columns = ['target','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment','gill-spacing',
'gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring',
'stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population',
'habitat']
##################################################################
#########################################
##################################################################
#########################################
def entropy(target_col):
  elements,counts = np.unique(target_col,return_counts = True)
  entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
  return entropy



Mounted at /content/drive/


In [ ]:
def InfoGain(data,split_attribute_name,target_name="target"):
  #Calculer l'entropie de l'ensemble de données total
  total_entropy = entropy(data[target_name])
  ##Calculer l'entropie du jeu de données
  #Calculer les valeurs et les décomptes correspondants pour l'attribut split
  vals,counts= np.unique(data[split_attribute_name],return_counts=True)
  #Calculer l'entropie pondérée
  Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) 
  for i in range(len(vals))])
  #Calculer le gain d'information
  Information_Gain = total_entropy - Weighted_Entropy
  return Information_Gain

In [ ]:
def ID3(data,originaldata,features,target_attribute_name="target",parent_node_class = None):
  #Définissez les critères d'arrêt --> Si l'un de ces critères est satisfait, nous voulons renvoyer un nœud feuille #
  #Si toutes les valeurs_cibles ont la même valeur, renvoie cette valeur
  if len(np.unique(data[target_attribute_name])) <= 1:
    return np.unique(data[target_attribute_name])[0]
    #Si l'ensemble de données est vide, renvoie la valeur de l'entité cible du mode dans l'ensemble de données d'origine
  elif len(data)==0:
    return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],
                                                                              return_counts=True)[1])]
  #Si l'espace de fonctions est vide, renvoyez la valeur de la fonction cible du mode du nœud parent direct --> Notez que
   #le nœud parent direct est le nœud qui a appelé l'exécution actuelle de l'algorithme ID3 et donc
   #la valeur de la caractéristique cible du mode est stockée dans la variable parent_node_class.
  elif len(features) ==0:
    return parent_node_class
  #Si rien de ce qui précède n'est vrai, faites pousser l'arbre!
  else:
  #Définir la valeur par défaut pour ce nœud --> La valeur de la fonction cible du mode du nœud actuel
    parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
    #unique_counts ndarray, facultatif :Le nombre de fois où chacune des valeurs uniques apparaît dans le tableau d'origine.
    ##################################################################
  ##############################################
  ############!!!!!!!!!Mettre en œuvre l'échantillonnage de sous-espace. Dessinez un certain nombre d'entités m = sqrt(p)!!!!!!!!#############
  ##################################################################
  #############################################
  features = np.random.choice(features,size=np.int(np.sqrt(len(features))),replace=False)
  #Sélectionnez l'entité qui divise le mieux l'ensemble de données
  item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] #Renvoie les valeurs de gain d'informations pour les entités du jeu de données
  best_feature_index = np.argmax(item_values)
  best_feature = features[best_feature_index]
  #Créer l'arborescence. La racine obtient le nom de la fonctionnalité (best_feature) avec le maximum d'informations
  #gain à la première manche
  tree = {best_feature:{}}
  #Supprimer la fonctionnalité avec le meilleur gain d'informations de l'espace des fonctionnalités
  features = [i for i in features if i != best_feature]
  #Développez une branche sous le nœud racine pour chaque valeur possible de la fonctionnalité du nœud racine
  for value in np.unique(data[best_feature]):
    value = value
    #Divisez le jeu de données le long de la valeur de l'entité avec le plus grand gain d'informations et créez ainsi des sous-ensembles de données
    sub_data = data.where(data[best_feature] == value).dropna()
    #Appelez l'algorithme ID3 pour chacun de ces sous-ensembles de données avec les nouveaux paramètres --> Ici, la récursivité entre en jeu !
    subtree = ID3(sub_data,dataset,features,target_attribute_name,parent_node_class)
    #Ajoutez le sous-arbre, développé à partir du sous-ensemble de données à l'arbre sous le nœud racine
    tree[best_feature][value] = subtree
  return(tree)



In [ ]:
def predict(query,tree,default = 'p'):
  for key in list(query.keys()):
    if key in list(tree.keys()):
      try:
        result = tree[key][query[key]]
      except:
        return default
      result = tree[key][query[key]]
      if isinstance(result,dict):
        return predict(query,result)
      else:
        return result


In [ ]:
##################################################################
#########################################
##################################################################
#########################################
def train_test_split(dataset):
  training_data = dataset.iloc[:round(0.75*len(dataset))].reset_index(drop=True)
  #Nous supprimons l'index respectivement renommons l'index
   #starting form 0, car nous ne voulons pas rencontrer d'erreurs concernant les étiquettes / index des lignes
  testing_data = dataset.iloc[round(0.75*len(dataset)):].reset_index(drop=True)
  return training_data,testing_data
training_data = train_test_split(dataset)[0]
testing_data = train_test_split(dataset)[1]
##################################################################
#########################################
##################################################################
#########################################
#######Entraîner le modèle Random Forest###########


In [ ]:
def RandomForest_Train(dataset,number_of_Trees):
  #Créer une liste dans laquelle les forêts individuelles sont stockées
  random_forest_sub_tree = []
  #Créer un certain nombre de n modèles
  for i in range(number_of_Trees):
    #Créer un certain nombre d'ensembles de données échantillonnés bootstrap à partir de l'ensemble de données d'origine
    bootstrap_sample = dataset.sample(frac=1,replace=True)
    #Créez un ensemble de données d'entraînement et de test en appelant la fonction train_test_split
    bootstrap_training_data = train_test_split(bootstrap_sample)[0]
    bootstrap_testing_data = train_test_split(bootstrap_sample)[1]
    #Développez un modèle d'arbre pour chacune des données d'apprentissage
    #Nous implémentons l'échantillonnage de sous-espace dans l'algorithme ID3 lui-même. Jetez donc un œil à l'algorithme ID3 ci-dessus !
    random_forest_sub_tree.append(ID3(bootstrap_training_data,bootstrap_training_data,
                      bootstrap_training_data.drop(labels=['target'],axis=1).columns))
  return random_forest_sub_tree
random_forest = RandomForest_Train(dataset,50)
#######Prédire une nouvelle instance de requête###########
def RandomForest_Predict(query,random_forest,default='p'):
  predictions = []
  for tree in random_forest:
    predictions.append(predict(query,tree,default))
  return sps.mode(predictions)[0][0]
query = testing_data.iloc[0,:].drop('target').to_dict()
query_target = testing_data.iloc[0,0]
print('target: ',query_target)
prediction = RandomForest_Predict(query,random_forest)

print('prediction: ',prediction)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


target:  e
prediction:  e


In [ ]:
#######Testez le modèle sur les données de test et renvoyez la précision###########
def RandomForest_Test(data,random_forest):
  data['predictions'] = None
  for i in range(len(data)):
    query = data.iloc[i,:].drop('target').to_dict()
    data.loc[i,'predictions'] = RandomForest_Predict(query,random_forest,default='p')
  accuracy = sum(data['predictions'] == data['target'])/len(data)*100
  #print('The prediction accuracy is: ',sum(data['predictions']== data['target'])/len(data)*100,'%')
  return accuracy
RandomForest_Test(testing_data,random_forest)


91.2358444116199

In [ ]:
##################################################################
############################################
##########Tracer la précision de la prédiction par rapport au nombre d'arbres dans les forêts aléatoires#############
##################################################################
############################################
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
fig = plt.figure(figsize=(15,10))

ax0 = fig.add_subplot(111)
accuracy = []
for i in range(1,11,1):
  random_forest = RandomForest_Train(dataset,i)
  accuracy.append(RandomForest_Test(testing_data,random_forest))
for i in range(10,110,10):
  random_forest = RandomForest_Train(dataset,i)
  accuracy.append(RandomForest_Test(testing_data,random_forest))
for i in range(100,1100,100):
  random_forest = RandomForest_Train(dataset,i)
  accuracy.append(RandomForest_Test(testing_data,random_forest))
print(accuracy)
ax0.plot(np.logspace(0,3,30),accuracy)
ax0.set_yticks(np.linspace(50,100,50))
ax0.set_title("Accuracy with respect to the numer of trees in the random forest")
ax0.set_xscale('log')
ax0.set_xlabel("Number of Trees")
ax0.set_ylabel('Accuracy(%)')
plt.show()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


#implémentation du random forests avec la librairie sklearn

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate
#Encode the feature values which are strings to integers
for label in dataset.columns:
  dataset[label] = LabelEncoder().fit(dataset[label]).transform(dataset[label])
X = dataset.drop(['target'],axis=1)
Y = dataset['target']
#Instantiate the model with 100 trees and entropy as splitting criteria
Random_Forest_model = RandomForestClassifier(n_estimators=100,criterion="entropy")
#Cross validation
accuracy = cross_validate(Random_Forest_model,X,Y,cv=10)['test_score']
print('The accuracy is: ',sum(accuracy)/len(accuracy)*100,'%')


The accuracy is:  100.0 %
